# Import Libraries and Data

In [136]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
pd.set_option("display.max_rows", 1000)

import sys
sys.path.append('../scripts')

from html_fiction_parser import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [142]:
df = pd.read_csv('../output/html_parse_2019-05-01.csv', index_col=[0])

index_to_exclude = [i for i in range(1,63)]
df = df.drop(index_to_exclude, axis=0)
df = df.reset_index()
df.shape

(2119, 9)

In [143]:
df.reset_index(inplace=True)
df['para_index'] = df['level_0']
df.head(10)


,level_0,index,para_index,tag,para,num_quotes_up_to_para,num_utterances,raw_utter_list,tokenized_sent,tokenized_sent_clean
0,0,0,0,narrative,By Jane Austen,0,0,[],[N],[N]
1,1,63,1,narrative,Chapter 1,0,0,[],[N],[N]
2,2,64,2,narrative,"It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.",0,0,[],[N],[N]
3,3,65,3,narrative,"However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.",0,0,[],[N],[N]
4,4,66,4,utterance,"""My dear Mr. Bennet,"" said his lady to him one day, ""have you heard that Netherfield Park is let at last?""",4,2,"['""My dear Mr. Bennet,""', '""have you heard that Netherfield Park is let at last?""']","""My dear Mr Bennet,"" [N] ""have you heard that Netherfield Park is let at last?""","My dear Mr Bennet, [N] have you heard that Netherfield Park is let at last?"
5,5,67,5,narrative,Mr. Bennet replied that he had not.,4,0,[],[N],[N]
6,6,68,6,utterance,"""But it is,"" returned she; ""for Mrs. Long has just been here, and she told me all about it.""",8,2,"['""But it is,""', '""for Mrs. Long has just been here, and she told me all about it.""']","""But it is,"" [N] ""for Mrs Long has just been here, and she told me all about it.""","But it is, [N] for Mrs Long has just been here, and she told me all about it."
7,7,69,7,narrative,Mr. Bennet made no answer.,8,0,[],[N],[N]
8,8,70,8,utterance,"""Do you not want to know who has taken it?"" cried his wife impatiently.",10,1,"['""Do you not want to know who has taken it?""']","""Do you not want to know who has taken it?"" [N]",Do you not want to know who has taken it? [N]
9,9,71,9,utterance,"""You want to tell me, and I have no objection to hearing it.""",12,1,"['""You want to tell me, and I have no objection to hearing it.""']","""You want to tell me, and I have no objection to hearing it.""","You want to tell me, and I have no objection to hearing it."


In [464]:
# df[(df['para'].apply(lambda x: is_quote_start_of_sent(x)==False)) & (df['tag']=='utterance')]

In [127]:
def find_utterances_raw(para):
    quotes = re.findall('"([^"]*)"', para)
    return ['"' + i + '"' for i in quotes]

def remove_period_from_honorifics(para):
    regex = '(Miss.|Mrs.|Mr.|Ms.)'
    for i in re.findall(regex, para, flags=re.IGNORECASE):
        para = para.replace(i, i[:-1])
    return para

def replace_narrative_w_mark(sent_list):
    new_sent_list = []
    for sent in sent_list:
        if ((get_num_quotes(sent))%2==0) & ((get_num_quotes(sent)!=0)):
            new_sent_list.append(sent)
        else:
            new_sent_list.append('[N]')
    return ' '.join(new_sent_list)

def tokenize_para(para):
    para = remove_period_from_honorifics(para)
#     regex = '((?![.\s])[^."]*(?:"[^"]*[^".]"[^."]*)*(?:"[^"]+\."|\.))'
#     regex = '((?![.\s])[^."]*(?:"[^"]*[^".]"[^."]*)*(?:"[^"]+\."|\.|"[^"]+\?"|"[^"]+\!"))'
    regex = r'(("[^"]*")|([^"]*))'
    results = [i[0] for i in re.findall(regex, para)]
    tag_texts = [i for i in results if i]
    output_string = replace_narrative_w_mark(tag_texts)
    return output_string


def tokenize_para_b(para):
    para = remove_period_from_honorifics(para)
#     regex = '((?![.\s])[^."]*(?:"[^"]*[^".]"[^."]*)*(?:"[^"]+\."|\.))'
#     regex = '((?![.\s])[^."]*(?:"[^"]*[^".]"[^."]*)*(?:"[^"]+\."|\.|"[^"]+\?"|"[^"]+\!"))'
    regex = r'(("[^"]*")|([^"]*))'
    results = [i[0] for i in re.findall(regex, para)]
    tag_texts = [i for i in results if i]
    output_string = ' '.join(tag_texts)
    return output_string


            

In [132]:
teststring0 = '“My dear Mr. Bennet,” said his lady to him one day, “have you heard that Netherfield Park is let at last?”'
print(teststring0)

“My dear Mr. Bennet,” said his lady to him one day, “have you heard that Netherfield Park is let at last?”


In [133]:
tokenize_para(teststring0)

'[N]'

In [128]:
teststring0 = '"Come here, child," cried her father as she appeared. "I have sent for you on an affair of importance. I understand that Mr. Collins has made you an offer of marriage. Is it true?" Elizabeth replied that it was. "Very well—and this offer of marriage you have refused?"'
print(teststring0)

"Come here, child," cried her father as she appeared. "I have sent for you on an affair of importance. I understand that Mr. Collins has made you an offer of marriage. Is it true?" Elizabeth replied that it was. "Very well—and this offer of marriage you have refused?"


In [129]:
tokenize_para(teststring0)

'"Come here, child," [N] "I have sent for you on an affair of importance. I understand that Mr Collins has made you an offer of marriage. Is it true?" [N] "Very well—and this offer of marriage you have refused?"'

In [130]:
teststring1 = '"My dear Mr. Bennet," replied his Ms. wife, "how can you be so tiresome! You must know that I am thinking of his marrying one of them."'
print(teststring1)

"My dear Mr. Bennet," replied his Ms. wife, "how can you be so tiresome! You must know that I am thinking of his marrying one of them."


In [131]:
tokenize_para(teststring1)

'"My dear Mr Bennet," [N] "how can you be so tiresome! You must know that I am thinking of his marrying one of them."'

In [19]:
find_utterances_raw(teststring1)

['"My dear Mr. Bennet,"',
 '"how can you be so tiresome! You must know that I am thinking of his marrying one of them."']

In [20]:
teststring2 = '"My dear Mr. Bennet," said his lady to him one day, "have you heard that Netherfield Park is let at last?"'
print(teststring2)

"My dear Mr. Bennet," said his lady to him one day, "have you heard that Netherfield Park is let at last?"


In [21]:
tokenize_para(teststring2)

'"My dear Mr Bennet," [N] "have you heard that Netherfield Park is let at last?"'

In [22]:
df['raw_utter_list'] = df['para'].apply(lambda x: find_utterances_raw(x))

In [23]:
df['tokenized_sent'] = df['para'].apply(lambda x: tokenize_para(x))

In [24]:
df.shape

(2181, 7)

# Cleaning Manual Labeled CSV

In [144]:
df_labeled = pd.read_csv('../../data/parsed-n-labeled-data/pnp-gutenberg-label-task-pride-and-prejudice-by-jane-austen.csv', index_col=[0])

In [145]:
df_labeled = df_labeled.reset_index()
df_labeled.columns = ['para_index', 'tag_old','para', 'position', 'speaker', 'type']

In [146]:
df_labeled['type'] = df_labeled['type'].replace(' c', 'c').replace('c ', 'c').fillna('')

In [147]:
df_labeled[df_labeled['position'].isnull()]

,para_index,tag_old,para,position,speaker,type
344,344,not,He was silent.,NaN,NaN,
1722,1722,not,Chapter 53,NaN,NaN,


In [148]:
df_labeled.loc[344, 'position'] = 'n'
df_labeled.loc[1722, 'position'] = 'n'

In [149]:
df_labeled['position'] = df_labeled['position'].apply(lambda x: x.lower())

In [150]:
df_labeled.loc[343]

para_index    343                                     
tag_old       dialogue-F                              
para          “How can you contrive to write so even?”
position      1                                       
speaker       NaN                                     
type                                                  
Name: 343, dtype: object

In [151]:
df_labeled['position'].value_counts()

n    785
1    689
2    507
3    99 
4    29 
5    8  
6    2  
Name: position, dtype: int64

In [152]:
df_labeled['type'].value_counts()

            1819
c           225 
letter      71  
quote       2   
emphasis    2   
Name: type, dtype: int64

In [153]:
def recount_convo(type_tag, position_tag):
    global convo_count
    
    if (type_tag == 'c') & (position_tag.lower()!='n'):
        convo_count += 1
        return 'c_{}'.format(convo_count)
    elif (type_tag == '') & (position_tag.lower()!='n'):
        return 'c_{}'.format(convo_count)
    elif (type_tag == 'letter'):
        return 'letter'
    elif (type_tag == 'quote'):
        return 'quote'
    else:
        return 'narrative'



In [154]:
convo_count = 0
df_labeled['new_type_tag'] = df_labeled.apply(lambda row: recount_convo(str(row['type']), str(row['position'])), axis=1)

In [155]:
df_labeled.head(10)

,para_index,tag_old,para,position,speaker,type,new_type_tag
0,0,not,By Jane Austen,n,NaN,,narrative
1,1,not,Chapter 1,n,NaN,,narrative
2,2,not,"It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.",n,NaN,,narrative
3,3,not,"However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.",n,NaN,,narrative
4,4,dialogue-F,"“My dear Mr. Bennet,” said his lady to him one day, “have you heard that Netherfield Park is let at last?”",1,mother,c,c_1
5,5,not,Mr. Bennet replied that he had not.,n,NaN,,narrative
6,6,dialogue-F,"“But it is,” returned she; “for Mrs. Long has just been here, and she told me all about it.”",1,mother,,c_1
7,7,not,Mr. Bennet made no answer.,n,NaN,,narrative
8,8,dialogue-F,“Do you not want to know who has taken it?” cried his wife impatiently.,1,mother,,c_1
9,9,dialogue-F,"“You want to tell me, and I have no objection to hearing it.”",2,father,,c_1


# Merging

In [156]:
df_merged = pd.merge(df, df_labeled, on='para_index', how='left')

In [157]:
df_merged.shape

(2119, 16)

In [158]:
df_merged.head(10)

,level_0,index,para_index,tag,para_x,num_quotes_up_to_para,num_utterances,raw_utter_list,tokenized_sent,tokenized_sent_clean,tag_old,para_y,position,speaker,type,new_type_tag
0,0,0,0,narrative,By Jane Austen,0,0,[],[N],[N],not,By Jane Austen,n,NaN,,narrative
1,1,63,1,narrative,Chapter 1,0,0,[],[N],[N],not,Chapter 1,n,NaN,,narrative
2,2,64,2,narrative,"It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.",0,0,[],[N],[N],not,"It is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.",n,NaN,,narrative
3,3,65,3,narrative,"However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.",0,0,[],[N],[N],not,"However little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.",n,NaN,,narrative
4,4,66,4,utterance,"""My dear Mr. Bennet,"" said his lady to him one day, ""have you heard that Netherfield Park is let at last?""",4,2,"['""My dear Mr. Bennet,""', '""have you heard that Netherfield Park is let at last?""']","""My dear Mr Bennet,"" [N] ""have you heard that Netherfield Park is let at last?""","My dear Mr Bennet, [N] have you heard that Netherfield Park is let at last?",dialogue-F,"“My dear Mr. Bennet,” said his lady to him one day, “have you heard that Netherfield Park is let at last?”",1,mother,c,c_1
5,5,67,5,narrative,Mr. Bennet replied that he had not.,4,0,[],[N],[N],not,Mr. Bennet replied that he had not.,n,NaN,,narrative
6,6,68,6,utterance,"""But it is,"" returned she; ""for Mrs. Long has just been here, and she told me all about it.""",8,2,"['""But it is,""', '""for Mrs. Long has just been here, and she told me all about it.""']","""But it is,"" [N] ""for Mrs Long has just been here, and she told me all about it.""","But it is, [N] for Mrs Long has just been here, and she told me all about it.",dialogue-F,"“But it is,” returned she; “for Mrs. Long has just been here, and she told me all about it.”",1,mother,,c_1
7,7,69,7,narrative,Mr. Bennet made no answer.,8,0,[],[N],[N],not,Mr. Bennet made no answer.,n,NaN,,narrative
8,8,70,8,utterance,"""Do you not want to know who has taken it?"" cried his wife impatiently.",10,1,"['""Do you not want to know who has taken it?""']","""Do you not want to know who has taken it?"" [N]",Do you not want to know who has taken it? [N],dialogue-F,“Do you not want to know who has taken it?” cried his wife impatiently.,1,mother,,c_1
9,9,71,9,utterance,"""You want to tell me, and I have no objection to hearing it.""",12,1,"['""You want to tell me, and I have no objection to hearing it.""']","""You want to tell me, and I have no objection to hearing it.""","You want to tell me, and I have no objection to hearing it.",dialogue-F,"“You want to tell me, and I have no objection to hearing it.”",2,father,,c_1


In [159]:
columns_i_want = ['para_index', 'para_x', 'num_utterances', 'raw_utter_list', 'tokenized_sent', 'tokenized_sent_clean', 'position', 'speaker', 'new_type_tag']
df_merged = df_merged[columns_i_want]

In [160]:
df_merged = df_merged.fillna('')

In [161]:
df_merged.columns

Index(['para_index', 'para_x', 'num_utterances', 'raw_utter_list',
       'tokenized_sent', 'tokenized_sent_clean', 'position', 'speaker',
       'new_type_tag'],
      dtype='object')

In [162]:
df_merged['chapter_tag'] = df_merged['para_x'].apply(lambda x: x if 'chapter ' in x.lower() else '')

In [163]:
chapter_dict = dict()
chapter_tag = ''
for i in df_merged.index:
    curr_chapter_tag = df_merged.loc[i]['chapter_tag']
    if curr_chapter_tag == '':
        if chapter_tag != '':
            chapter_dict[i] = chapter_tag
        else:
            chapter_tag = curr_chapter_tag
            chapter_dict[i] = chapter_tag
    else: 
        if chapter_tag == curr_chapter_tag:
            chapter_dict[i] = chapter_tag
        else:
            chapter_tag = curr_chapter_tag
            chapter_dict[i] = chapter_tag



In [164]:
df_merged['new_chapter_tag'] = list(x[1] for x in chapter_dict.items())

In [165]:
def is_utterance(sent):
    return get_num_quotes(sent)%2==0


def tag_tokens(sent_list):
    new_list = []
    for sent in sent_list:
        if is_utterance(sent):
            new_list.append((sent, ))

In [166]:
new_label_dict = dict()
count = 0 
for j in df_merged.index:
    tuple_i_want = (j,) \
                + tuple(list(df_merged.loc[j][['position', 'speaker', 'new_type_tag', 'new_chapter_tag']].values)) \
                + (tokenize_para(df_merged.loc[j]['para_x']),)
    new_label_dict[count] = tuple_i_want
    count += 1     

In [167]:
df_final = pd.DataFrame(list(new_label_dict.items()), columns=['sent_index', 'tag'])

In [168]:
df_final['para_index'] = df_final['tag'].apply(lambda x: x[0])
df_final['position_tag'] = df_final['tag'].apply(lambda x: x[1])
df_final['speaker_tag'] = df_final['tag'].apply(lambda x: x[2])
df_final['type_tag'] = df_final['tag'].apply(lambda x: x[3])
df_final['chapter_tag'] = df_final['tag'].apply(lambda x: x[4])
df_final['tokenized_sent'] = df_final['tag'].apply(lambda x: x[5])


In [169]:
df_final.head(100)

,sent_index,tag,para_index,position_tag,speaker_tag,type_tag,chapter_tag,tokenized_sent
0,0,"(0, n, , narrative, , [N])",0,n,,narrative,,[N]
1,1,"(1, n, , narrative, Chapter 1, [N])",1,n,,narrative,Chapter 1,[N]
2,2,"(2, n, , narrative, Chapter 1, [N])",2,n,,narrative,Chapter 1,[N]
3,3,"(3, n, , narrative, Chapter 1, [N])",3,n,,narrative,Chapter 1,[N]
4,4,"(4, 1, mother, c_1, Chapter 1, ""My dear Mr Bennet,"" [N] ""have you heard that Netherfield Park is let at last?"")",4,1,mother,c_1,Chapter 1,"""My dear Mr Bennet,"" [N] ""have you heard that Netherfield Park is let at last?"""
5,5,"(5, n, , narrative, Chapter 1, [N])",5,n,,narrative,Chapter 1,[N]
6,6,"(6, 1, mother, c_1, Chapter 1, ""But it is,"" [N] ""for Mrs Long has just been here, and she told me all about it."")",6,1,mother,c_1,Chapter 1,"""But it is,"" [N] ""for Mrs Long has just been here, and she told me all about it."""
7,7,"(7, n, , narrative, Chapter 1, [N])",7,n,,narrative,Chapter 1,[N]
8,8,"(8, 1, mother, c_1, Chapter 1, ""Do you not want to know who has taken it?"" [N])",8,1,mother,c_1,Chapter 1,"""Do you not want to know who has taken it?"" [N]"
9,9,"(9, 2, father, c_1, Chapter 1, ""You want to tell me, and I have no objection to hearing it."")",9,2,father,c_1,Chapter 1,"""You want to tell me, and I have no objection to hearing it."""


In [170]:
index_to_exclude = df_final[df_final['tokenized_sent'].apply(lambda x: x=='[N]')].index

In [171]:
df_final = df_final.drop(index_to_exclude, axis=0)

In [172]:
df_final = df_final.reset_index()
del df_final['index']

In [173]:
df_final.shape

(1290, 8)

In [174]:
label_dict = dict()

conver_tag = ''
position_tag = 'n'

for i in df_final.index:
    curr_position_tag = df_final.loc[i]['position_tag']
    curr_conver_tag = df_final.loc[i]['type_tag']
    if (curr_conver_tag == conver_tag) & (curr_position_tag!='n'):
        if curr_position_tag == position_tag:
            label_dict[i] = 'part'
            position_tag = curr_position_tag
            conver_tag = curr_conver_tag
        else:
            label_dict[i] = 'response'
            position_tag = curr_position_tag
            conver_tag = curr_conver_tag
    else: 
        label_dict[i] = 'not'
        position_tag = curr_position_tag
        conver_tag = curr_conver_tag    

In [175]:
df_final['label'] = list(x[1] for x in label_dict.items())

In [176]:
df_final.head(100)

,sent_index,tag,para_index,position_tag,speaker_tag,type_tag,chapter_tag,tokenized_sent,label
0,4,"(4, 1, mother, c_1, Chapter 1, ""My dear Mr Bennet,"" [N] ""have you heard that Netherfield Park is let at last?"")",4,1,mother,c_1,Chapter 1,"""My dear Mr Bennet,"" [N] ""have you heard that Netherfield Park is let at last?""",not
1,6,"(6, 1, mother, c_1, Chapter 1, ""But it is,"" [N] ""for Mrs Long has just been here, and she told me all about it."")",6,1,mother,c_1,Chapter 1,"""But it is,"" [N] ""for Mrs Long has just been here, and she told me all about it.""",part
2,8,"(8, 1, mother, c_1, Chapter 1, ""Do you not want to know who has taken it?"" [N])",8,1,mother,c_1,Chapter 1,"""Do you not want to know who has taken it?"" [N]",part
3,9,"(9, 2, father, c_1, Chapter 1, ""You want to tell me, and I have no objection to hearing it."")",9,2,father,c_1,Chapter 1,"""You want to tell me, and I have no objection to hearing it.""",response
4,11,"(11, 1, , c_1, Chapter 1, ""Why, my dear, you must know, Mrs Long says that Netherfield is taken by a young man of large fortune from the north of England; that he came down on Monday in a chaise and four to see the place, and was so much delighted with it, that he agreed with Mr Morris immediately; that he is to take possession before Michaelmas, and some of his servants are to be in the house by the end of next week."")",11,1,,c_1,Chapter 1,"""Why, my dear, you must know, Mrs Long says that Netherfield is taken by a young man of large fortune from the north of England; that he came down on Monday in a chaise and four to see the place, and was so much delighted with it, that he agreed with Mr Morris immediately; that he is to take possession before Michaelmas, and some of his servants are to be in the house by the end of next week.""",response
5,12,"(12, 2, , c_1, Chapter 1, ""What is his name?"")",12,2,,c_1,Chapter 1,"""What is his name?""",response
6,13,"(13, 1, , c_1, Chapter 1, ""Bingley."")",13,1,,c_1,Chapter 1,"""Bingley.""",response
7,14,"(14, 2, , c_1, Chapter 1, ""Is he married or single?"")",14,2,,c_1,Chapter 1,"""Is he married or single?""",response
8,15,"(15, 1, , c_1, Chapter 1, ""Oh! Single, my dear, to be sure! A single man of large fortune; four or five thousand a year. What a fine thing for our girls!"")",15,1,,c_1,Chapter 1,"""Oh! Single, my dear, to be sure! A single man of large fortune; four or five thousand a year. What a fine thing for our girls!""",response
9,16,"(16, 2, , c_1, Chapter 1, ""How so? How can it affect them?"")",16,2,,c_1,Chapter 1,"""How so? How can it affect them?""",response


In [177]:
df_final.shape

(1290, 9)

chapters  19–26  asthe  test  set,  chapters  27–33  as  the  developmentset, and the remaining 46 chapters as the trainingset

In [178]:
test_set_chapters = ['Chapter {}'.format(x) for x in range(19,27)]
validation_set_chapters = ['Chapter {}'.format(x) for x in range(27,34)]


In [179]:
df_test = df_final[df_final['chapter_tag'].apply(lambda x: True if x in test_set_chapters else False)]

In [180]:
df_validation = df_final[df_final['chapter_tag'].apply(lambda x: True if x in validation_set_chapters else False)]

In [181]:
def custom_train_test_split(field):
    if field in test_set_chapters:
        return 'test'
    elif field in validation_set_chapters:
        return 'validation'
    else:
        return 'train'

In [182]:
df_final['split_tag'] = df_final['chapter_tag'].apply(lambda x: custom_train_test_split(x))

In [183]:
df_final['split_tag'].value_counts()

train         1036
test          129 
validation    125 
Name: split_tag, dtype: int64

In [184]:
pair_list = []
for i in df_final.index:
    first_utter = df_final.loc[i]['tokenized_sent']
    first_utter_label = df_final.loc[i]['label']
    first_utter_split = df_final.loc[i]['split_tag']
    if i != 0:
        if first_utter_label == 'response':
            prev_utter = df_final.loc[i-1]['tokenized_sent'] 
            pair_list.append((prev_utter, first_utter, 'response', first_utter_split))
        elif first_utter_label == 'part':
            prev_utter = df_final.loc[i-1]['tokenized_sent'] 
            pair_list.append((prev_utter, first_utter, 'part', first_utter_split))

        else:
            pair_list.append((prev_utter, first_utter, 'not_pair', first_utter_split))


In [185]:
pair_list

[('"My dear Mr Bennet," [N] "have you heard that       Netherfield Park is let at last?"',
  '"But it is," [N] "for Mrs Long has just been here, and she told       me all about it."',
  'part',
  'train'),
 ('"But it is," [N] "for Mrs Long has just been here, and she told       me all about it."',
  '"Do you not want to know who has taken it?" [N]',
  'part',
  'train'),
 ('"Do you not want to know who has taken it?" [N]',
  '"You want to tell me, and I have no objection to hearing it."',
  'response',
  'train'),
 ('"You want to tell me, and I have no objection to hearing it."',
  '"Why, my dear, you must know, Mrs Long says that Netherfield is taken by       a young man of large fortune from the north of England; that he came down       on Monday in a chaise and four to see the place, and was so much delighted       with it, that he agreed with Mr Morris immediately; that he is to take       possession before Michaelmas, and some of his servants are to be in the       house by the en

In [186]:
df_final_pair = pd.DataFrame()

In [187]:
df_final_pair['utter_1'] = [x[0] for x in pair_list]
df_final_pair['utter_2'] = [x[1] for x in pair_list]
df_final_pair['label'] = [x[2] for x in pair_list]
df_final_pair['split_tag'] = [x[3] for x in pair_list]


In [188]:
df_final_pair.head(100)

,utter_1,utter_2,label,split_tag
0,"""My dear Mr Bennet,"" [N] ""have you heard that Netherfield Park is let at last?""","""But it is,"" [N] ""for Mrs Long has just been here, and she told me all about it.""",part,train
1,"""But it is,"" [N] ""for Mrs Long has just been here, and she told me all about it.""","""Do you not want to know who has taken it?"" [N]",part,train
2,"""Do you not want to know who has taken it?"" [N]","""You want to tell me, and I have no objection to hearing it.""",response,train
3,"""You want to tell me, and I have no objection to hearing it.""","""Why, my dear, you must know, Mrs Long says that Netherfield is taken by a young man of large fortune from the north of England; that he came down on Monday in a chaise and four to see the place, and was so much delighted with it, that he agreed with Mr Morris immediately; that he is to take possession before Michaelmas, and some of his servants are to be in the house by the end of next week.""",response,train
4,"""Why, my dear, you must know, Mrs Long says that Netherfield is taken by a young man of large fortune from the north of England; that he came down on Monday in a chaise and four to see the place, and was so much delighted with it, that he agreed with Mr Morris immediately; that he is to take possession before Michaelmas, and some of his servants are to be in the house by the end of next week.""","""What is his name?""",response,train
5,"""What is his name?""","""Bingley.""",response,train
6,"""Bingley.""","""Is he married or single?""",response,train
7,"""Is he married or single?""","""Oh! Single, my dear, to be sure! A single man of large fortune; four or five thousand a year. What a fine thing for our girls!""",response,train
8,"""Oh! Single, my dear, to be sure! A single man of large fortune; four or five thousand a year. What a fine thing for our girls!""","""How so? How can it affect them?""",response,train
9,"""How so? How can it affect them?""","""My dear Mr Bennet,"" [N] ""how can you be so tiresome! You must know that I am thinking of his marrying one of them.""",response,train


In [189]:
df_final_pair.shape

(1289, 4)

In [1423]:
# df_final['tokenized_sent_clean'] = df_final['tokenized_sent'].apply(lambda x: re.sub('"', '', x))

In [1424]:
# df_final.head(50)

,sent_index,tag,para_index,position_tag,speaker_tag,type_tag,tokenized_sent,chapter_tag,label,split_tag,tokenized_sent_clean
0,4,"(4, 1, mother, c_1, Chapter 1, ""My dear Mr Bennet,"" [N] ""have you heard that Netherfield Park is let at last?"")",4,1,mother,c_1,"""My dear Mr Bennet,"" [N] ""have you heard that Netherfield Park is let at last?""",Chapter 1,not,train,"My dear Mr Bennet, [N] have you heard that Netherfield Park is let at last?"
1,6,"(6, 1, mother, c_1, Chapter 1, ""But it is,"" [N] ""for Mrs Long has just been here, and she told me all about it."")",6,1,mother,c_1,"""But it is,"" [N] ""for Mrs Long has just been here, and she told me all about it.""",Chapter 1,part,train,"But it is, [N] for Mrs Long has just been here, and she told me all about it."
2,8,"(8, 1, mother, c_1, Chapter 1, ""Do you not want to know who has taken it?"" [N])",8,1,mother,c_1,"""Do you not want to know who has taken it?"" [N]",Chapter 1,part,train,Do you not want to know who has taken it? [N]
3,9,"(9, 2, father, c_1, Chapter 1, ""You want to tell me, and I have no objection to hearing it."")",9,2,father,c_1,"""You want to tell me, and I have no objection to hearing it.""",Chapter 1,response,train,"You want to tell me, and I have no objection to hearing it."
4,11,"(11, 1, , c_1, Chapter 1, ""Why, my dear, you must know, Mrs Long says that Netherfield is taken by a young man of large fortune from the north of England; that he came down on Monday in a chaise and four to see the place, and was so much delighted with it, that he agreed with Mr Morris immediately; that he is to take possession before Michaelmas, and some of his servants are to be in the house by the end of next week."")",11,1,,c_1,"""Why, my dear, you must know, Mrs Long says that Netherfield is taken by a young man of large fortune from the north of England; that he came down on Monday in a chaise and four to see the place, and was so much delighted with it, that he agreed with Mr Morris immediately; that he is to take possession before Michaelmas, and some of his servants are to be in the house by the end of next week.""",Chapter 1,response,train,"Why, my dear, you must know, Mrs Long says that Netherfield is taken by a young man of large fortune from the north of England; that he came down on Monday in a chaise and four to see the place, and was so much delighted with it, that he agreed with Mr Morris immediately; that he is to take possession before Michaelmas, and some of his servants are to be in the house by the end of next week."
5,12,"(12, 2, , c_1, Chapter 1, ""What is his name?"")",12,2,,c_1,"""What is his name?""",Chapter 1,response,train,What is his name?
6,13,"(13, 1, , c_1, Chapter 1, ""Bingley."")",13,1,,c_1,"""Bingley.""",Chapter 1,response,train,Bingley.
7,14,"(14, 2, , c_1, Chapter 1, ""Is he married or single?"")",14,2,,c_1,"""Is he married or single?""",Chapter 1,response,train,Is he married or single?
8,15,"(15, 1, , c_1, Chapter 1, ""Oh! Single, my dear, to be sure! A single man of large fortune; four or five thousand a year. What a fine thing for our girls!"")",15,1,,c_1,"""Oh! Single, my dear, to be sure! A single man of large fortune; four or five thousand a year. What a fine thing for our girls!""",Chapter 1,response,train,"Oh! Single, my dear, to be sure! A single man of large fortune; four or five thousand a year. What a fine thing for our girls!"
9,16,"(16, 2, , c_1, Chapter 1, ""How so? How can it affect them?"")",16,2,,c_1,"""How so? How can it affect them?""",Chapter 1,response,train,How so? How can it affect them?


In [190]:
df_final_pair.to_csv('../../data/parsed-n-labeled-data/labeled-utterance-pair-final-020519.csv')